<a href="https://colab.research.google.com/github/jc533/Song_data_game/blob/attackofthetitan-patch-1/data_game_preprocess.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# method 1

# import

In [ ]:
import sys
!cp ../input/rapids/rapids.0.14.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.10/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.10"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/
import warnings
warnings.filterwarnings('ignore')

'cp' is not recognized as an internal or external command,
operable program or batch file.
The system cannot find the path specified.
'cp' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
import numba
numba.set_num_threads(24)
#numba.set_num_threads(8)

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import MultiLabelBinarizer

# read file

In [ ]:
#path = '/kaggle/input/datagame/datagame-2023'
#path = "/content/drive/MyDrive/ML/datagame-2023"
path = 'C:/Users/ethan/Downloads/datagame-2023'
train = pd.read_parquet(f'{path}/label_train_source.parquet')
test = pd.read_parquet(f'{path}/label_test_source.parquet')
target = pd.read_parquet(f'{path}/label_train_target.parquet')
composer = pd.read_parquet(f'{path}/meta_song_composer.parquet')
genre = pd.read_parquet(f'{path}/meta_song_genre.parquet')
lyricist = pd.read_parquet(f'{path}/meta_song_lyricist.parquet')
producer = pd.read_parquet(f'{path}/meta_song_producer.parquet')
titletext = pd.read_parquet(f'{path}/meta_song_titletext.parquet')
song = pd.read_parquet(f'{path}/meta_song.parquet')


# Custom map


In [ ]:
label_encoder = LabelEncoder()

In [ ]:
tmp = pd.DataFrame()
tmp["song_id"] = song["song_id"]
label_encoder.fit(tmp["song_id"])
unmapping = dict(zip(label_encoder.transform(label_encoder.classes_,),label_encoder.classes_))
mapping = dict(zip(label_encoder.classes_,label_encoder.transform(label_encoder.classes_,)))

# Feature engineer genre

In [ ]:
mlb = MultiLabelBinarizer(classes=genre['genre_id'].unique())
genre_df = pd.DataFrame(mlb.fit_transform(genre['genre_id']),columns=genre['genre_id'].unique())
genre_df = pd.concat([genre['song_id'], genre_df], axis=1)
genre_df = genre_df.groupby('song_id').sum().reset_index()

# merge data


In [ ]:
song.drop(['song_length', 'album_id', 'album_month'], axis=1, inplace=True)
train_d = train.drop(['unix_played_at', 'play_status', 'login_type',], axis=1, )
test_d = test.drop(['unix_played_at', 'play_status', 'login_type', ], axis=1, )
target_d = target.drop(['unix_played_at', 'play_status', 'login_type', ], axis=1, )

In [ ]:
#song_data = pd.concat([train, target])
#song_data = song_data.sort_values(by=['session_id', 'listening_order'])
#song_data = pd.merge(train, target, on='session_id', how='inner')
song_data = pd.merge(train_d, song, on='song_id', how='left')
#song_data = pd.merge(song_data, composer, on='song_id', how='left')
song_data = pd.merge(song_data, genre_df, on='song_id', how='left')
#song_data = pd.merge(song_data, lyricist, on='song_id', how='left')
#song_data = pd.merge(song_data, producer, on='song_id', how='left')
#song_data = pd.merge(song_data, titletext, on='song_id', how='left')
song_vali = pd.merge(target_d, song, on='song_id', how='left')
#song_vali = pd.merge(song_vali, composer, on='song_id', how='left')
song_vali = pd.merge(song_vali, genre_df, on='song_id', how='left')
#song_vali = pd.merge(song_vali, lyricist, on='song_id', how='left')
#song_vali = pd.merge(song_vali, producer, on='song_id', how='left')
#song_vali = pd.merge(song_vali, titletext, on='song_id', how='left')


In [ ]:
print(song_data.shape)

(11445180, 43)


In [ ]:
song_test = pd.merge(test_d, song, on='song_id', how='left')
#song_test = pd.merge(song_test, composer, on='song_id', how='left', validate='m:1')
song_test = pd.merge(song_test, genre_df, on='song_id', how='left')
#song_test = pd.merge(song_test, lyricist, on='song_id', how='left')
#song_test = pd.merge(song_test, producer, on='song_id', how='left')
#song_test = pd.merge(song_test, titletext, on='song_id', how='left')
print(test.shape)
print(song_test.shape)

(2861280, 6)
(2861280, 43)


In [ ]:
song_data.drop_duplicates(subset=['session_id', 'song_id', 'listening_order'], inplace=True)
song_vali.drop_duplicates(subset=['session_id', 'song_id', 'listening_order'], inplace=True)
print(song_data.shape)
print(song_vali.shape)

(11445180, 43)
(2861295, 43)


In [ ]:
song_data.drop(['listening_order'], axis=1, inplace=True)
song_vali.drop(['listening_order'], axis=1, inplace=True)
song_test.drop(['listening_order'], axis=1, inplace=True)

In [ ]:
for i in song_data.select_dtypes(include=['object']).columns:
  song_data[i][song_data[i].isnull()] = 'unknown'
song_data = song_data.fillna(value=0)

for i in song_vali.select_dtypes(include=['object']).columns:
  song_vali[i][song_vali[i].isnull()] = 'unknown'
song_vali = song_vali.fillna(value=0)


for i in song_test.select_dtypes(include=['object']).columns:
  song_test[i][song_test[i].isnull()] = 'unknown'
song_test = song_test.fillna(value=0)


encol = [ x for x in song_data.columns if x not in ["song_id"] ]
for i in encol:
  song_data[i] = label_encoder.fit_transform(song_data[i])
encol = [ x for x in song_vali.columns if x not in ["song_id"] ]
for i in encol:
  song_vali[i] = label_encoder.fit_transform(song_vali[i])
encol = [ x for x in song_test.columns if x not in ["song_id","session_id"] ]
for i in encol:
  song_test[i] = label_encoder.fit_transform(song_test[i])

label_encoder.fit(song_test["session_id"])

sess_unmapping = dict(zip(label_encoder.transform(label_encoder.classes_,),label_encoder.classes_))
song_test["session_id"] = label_encoder.fit_transform(song_test["session_id"])

song_data["song_id"] = song_data["song_id"].map(mapping)
song_vali["song_id"] = song_vali["song_id"].map(mapping)
song_test["song_id"] = song_test["song_id"].map(mapping)

song_data.reset_index(drop=True)
song_vali.reset_index(drop=True)
song_test.reset_index(drop=True)

,session_id,song_id,artist_id,language_id,b856b6781d370a3645c6dde0c20b3597,2eabe9f164346c7b3ff1bd23078f483e,bb3d7b04b67d5aeb5ab145bdd70750da,ce4db56f6a48426643b08038139a8a75,43244ec4c30a0dad837d982892bc0c05,bb737ea7450c3abbab1ff613a2c6309e,...,1f4b914a79eb2bb01c2ea694af626625,03c358e326d99a3863e044c5d8e9fb50,d36204de09c0c6084d55b1f484a23773,f6f02545c1905a8c72c5bf006579996e,ee1696df08c0ac6005e3b9442cdbbded,1a619fcf3adfd91711699b7e2cf2c367,80117354556efbf237f0020bea2f7e42,c3ad773a264597f9f46c2d666e1a8b50,74694a488312db91fcd56818fed8b3a6,5455e1699025b025a3523aba4719e818
0,118,271639,5333,12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,118,589665,4126,12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,118,331055,165,8,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,118,297816,1693,12,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,118,759365,4379,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2861275,142898,80547,48613,8,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2861276,142898,659924,48613,8,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2861277,142898,626741,48613,8,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2861278,142898,909527,48613,8,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
print(song_data.shape[0]/20)
print(song_vali.shape[0]/5)
print(song_test.shape[0]/20)

572259.0
572259.0
143064.0


In [ ]:
training = song_data.groupby('session_id')
targeting = song_vali.groupby('session_id')
testing = song_test.groupby('session_id')

In [ ]:
print(song_data.isna().sum())
print(song_vali.isna().sum())
print(song_test.isna().sum())

session_id                          0
song_id                             0
artist_id                           0
language_id                         0
b856b6781d370a3645c6dde0c20b3597    0
2eabe9f164346c7b3ff1bd23078f483e    0
bb3d7b04b67d5aeb5ab145bdd70750da    0
ce4db56f6a48426643b08038139a8a75    0
43244ec4c30a0dad837d982892bc0c05    0
bb737ea7450c3abbab1ff613a2c6309e    0
5f2a134d2289a8a3de6663ee8e248c8a    0
6ea61b86b8fff0e3a05bc73ea4eaf21f    0
1d7c8bb87dcc1457ed90240c06f9ebdf    0
abdc0a50aff67c591737bc6f57a36e09    0
5d7d428b81ab3429f189bbc642548b53    0
a2a4a0943fe2c5fe6891d9c34dca906f    0
a3b51d979053a7f49511b9ab72ee8878    0
e1d5802bb4e0f6ab79b2b9f4a5ff924b    0
d9ab2ffb929a854e208efaf5297b7cf8    0
c1d8909eff8a140e7bd0c11508f76dbe    0
65aeeca3341ca1c6a2ed774aa4e22add    0
09994f2b6bf22721e7379df4e22b1041    0
ed514a72b48a9d15df7bc4c25eac2c67    0
56203098bef705a44fee44ac0d9b7ef2    0
23ae47469cc0a10de43c453ab59c87b4    0
4c4ce52d6f6a0e495c407b584ef3e020    0
7ed5eec2ea6f

# Preprocess


In [ ]:
sessid = []
itm = testing.groups.items()
for sess_id, group_indices in itm:
    sessid.append(sess_id)
sessid = pd.Series(sessid)

In [ ]:
t = {}
k=0
s=100000
#s=1
size = len(song_data)/20
itm = training.groups.items()
for sess_id, group_indices in itm:
    tmp = song_data.loc[group_indices]
    tmp.drop(columns = ['session_id'],inplace=True)
    tmp = tmp.to_numpy().flatten()
    tmp = tmp.tolist()
    t[sess_id] = tmp
    if k%s==0 and k>=s:
      print(k,k/size*100)
    k+=1

100000 17.474605030239804
200000 34.94921006047961
300000 52.423815090719415
400000 69.89842012095922
500000 87.37302515119903


In [ ]:
te = {}
k=0
s=100000
#s=1
size = len(song_test)/20
itm = testing.groups.items()
for sess_id, group_indices in itm:
    tmp = song_test.loc[group_indices]
    tmp.drop(columns = ['session_id'],inplace=True)
    tmp = tmp.to_numpy().flatten()
    tmp = tmp.tolist()
    te[sess_id] = tmp
    if k%s==0 and k>=s:
      print(k,k/size*100)
    k+=1

100000 69.89878655706538


In [ ]:
v = {}
k=0
s=100000
# s=1
size = len(song_vali)/5
itm = targeting.groups.items()


for sess_id, group_indices in itm:
    tmp = song_vali.loc[group_indices, 'song_id'].to_numpy()
    tmp = tmp.tolist()
    v[sess_id] = tmp
    if k%s==0 and k>=s:
      print(k,k/size*100)
    k+=1


100000 17.474605030239804
200000 34.94921006047961
300000 52.423815090719415
400000 69.89842012095922
500000 87.37302515119903


In [ ]:
df_t = pd.DataFrame.from_dict(t)
df_te = pd.DataFrame(te)
df_v = pd.DataFrame.from_dict(v)
df_v

,0,1,2,3,4,5,6,7,8,9,...,572249,572250,572251,572252,572253,572254,572255,572256,572257,572258
0,714573,827206,649931,993467,987875,101458,481537,879764,321975,92839,...,53839,776460,563546,675403,502061,243958,727401,154284,785711,865600
1,66954,523535,541297,912199,617147,222834,86305,167308,183542,745216,...,367695,901098,563546,760641,478645,647039,890711,409034,685466,313485
2,4081,954908,792701,993467,987875,910490,667533,879764,559588,842457,...,327633,264931,563546,817777,898521,374691,786880,680838,842585,408659
3,140308,103310,317012,912199,617147,727800,973667,958286,146397,538151,...,692564,369117,563546,212330,386071,239341,985016,690473,973141,42347
4,862634,360635,867247,993467,723216,204125,27481,167308,918698,412091,...,569619,219772,563546,694460,990338,340380,985016,7274,842585,365513


In [ ]:
X = df_t.transpose()
y = df_v.transpose()
testX = df_te.transpose()

# Train

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X, y, test_size = 0.25, random_state = 0)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn = KNeighborsRegressor(n_neighbors=5, weights='distance', leaf_size=30, p=2, n_jobs=12)
knn = knn.fit(X_train, Y_train)
knn_pred = knn.predict(testX)

In [ ]:
def unmapping_fun(x):
  if x not in unmapping:
    t = np.random.randint(0, len(unmapping))
    return unmapping[t]
  else:
    return unmapping[x]

In [ ]:
knn_pred = pd.DataFrame(knn_pred).astype(int)
for i in knn_pred.columns:
  knn_pred[i] = knn_pred[i].map(unmapping_fun)

In [ ]:
session_test = sessid.map(sess_unmapping)
knn_pred = pd.concat([session_test, knn_pred], axis=1)
knn_pred.to_csv(f'{path}/subknn.csv', header=['session_id', 'top1', 'top2', 'top3', 'top4', 'top5'], index=False)

# method 2

## import
download cudf for speed up pandas

In [ ]:
import sys
!cp ../input/rapids/rapids.0.14.0 /opt/conda/envs/rapids.tar.gz
!cd /opt/conda/envs/ && tar -xzvf rapids.tar.gz > /dev/null
sys.path = ["/opt/conda/envs/rapids/lib/python3.10/site-packages"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib/python3.10"] + sys.path
sys.path = ["/opt/conda/envs/rapids/lib"] + sys.path
!cp /opt/conda/envs/rapids/lib/libxgboost.so /opt/conda/lib/
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import matplotlib as plt

cp: cannot stat '../input/rapids/rapids.0.14.0': No such file or directory
tar (child): rapids.tar.gz: Cannot open: No such file or directory
tar (child): Error is not recoverable: exiting now
tar: Child returned status 2
tar: Error is not recoverable: exiting now
cp: cannot stat '/opt/conda/envs/rapids/lib/libxgboost.so': No such file or directory


## read file

In [ ]:
file_path = "/kaggle/input/datagame/datagame-2023/"
train = pd.read_parquet(f"{file_path}/label_train_source.parquet")
test = pd.read_parquet(f"{file_path}/label_test_source.parquet")
testg = test.groupby('session_id')
traing = train.groupby('session_id')
import cudf as pd

train = pd.read_parquet(f"{file_path}/label_train_source.parquet")
test = pd.read_parquet(f"{file_path}/label_test_source.parquet")
target = pd.read_parquet(f"{file_path}/label_train_target.parquet")
meta_song = pd.read_parquet(f"{file_path}/meta_song.parquet")
trainR = train["song_id"]
targetR = target["song_id"]
testR = test["song_id"]
rank = pd.concat([trainR,targetR,testR])

## create rank
for random

In [ ]:
r = rank.value_counts()
rank_result = r.rank(ascending=False, method='dense')
ranked = pd.DataFrame({'Value': r.index, 'Count': r.values, 'Rank': rank_result.values})


## model

In [ ]:
r = {}
c = 0
def model45a():
    size = len(test)/20
    fill = ranked["Value"][:5].to_numpy()
    k = 0
    c=0
    s=10000
    itm = testg.groups.items()
    for sess_id, group_indices in itm:
        tmp = test.loc[group_indices, "song_id"].to_numpy()
        temp = pd.Series(tmp).value_counts()
        temp = temp[temp>1].index[:5]
        length = len(temp)
        if length<5:
#             print(fill)
#             print(temp)
            n = 5-length
            c += n

            rnd = ranked[~ranked["Value"].isin(temp)]
            fill = ranked.sample(n)["Value"].to_numpy()
            temp = np.append(temp.to_numpy(),fill)
        r[sess_id] = temp
        if k%s==0 and k>=s:
#             break
            print(k,k/size*100)
        k+=1
%time model45a()
sub = pd.DataFrame.from_dict(r).transpose()
sub.to_csv("./sub.csv",header=['session_id', 'top1', 'top2', 'top3', 'top4', 'top5'])
sub

10000 6.989878655706537
20000 13.979757311413074
30000 20.96963596711961
40000 27.95951462282615
50000 34.94939327853269
60000 41.93927193423922
70000 48.929150589945756
80000 55.9190292456523
90000 62.90890790135884
100000 69.89878655706538
110000 76.88866521277191
120000 83.87854386847845
130000 90.86842252418498
140000 97.85830117989151
CPU times: user 51min 24s, sys: 19min 50s, total: 1h 11min 14s
Wall time: 1h 10min 34s
0


,0,1,2,3,4
8,b429598c3eec37ec5755f28949bd6e73,97d3de77e15ff6140529932da474ac0c,2ef66cf1db93024c492dac62bf6fc5e2,dd612f48085759fc2ce953ecffd17793,464cf913b18246d713ad20ae37e02cd1
9,3c8127d8f24589c12336d5083ba9c492,555b8220459e95669be059c90c355d64,31524bfa5a1e9a8cc9ad3a54b5d5d1fc,0f4ecca1dc979abdcc4760553be15bf0,bb9c470243d89b63fd92f4b8d822aa22
18,43ff50171dcf94f2ad0e775ef1a6c707,66e6cbab82630d69acca2115157fbcba,52e71ff87ac2dad3e11b807b481b9c5b,134c6056b5ac337ecdd22a734f0f15d8,9efd4fe6838c54c5e9a3fef99c4eb54e
19,6c33e2032630dcafde3a4fd5ae196dc2,45a351c51fe0d8274f0303266a9e0c7d,56a3d0be4364e8812518eeb22d0519ca,81e9fe5eb6f896d703936ec14b7795e3,2054dfc2ada3c4babe0c488af86a38d3
28,7873629643d3d71732b639e37c381d00,febdbc34c0543eb8c2106d8831929d4d,adf1110735db80f00fc3cc9283f6ce64,db02ec2cada684ecd1be2fc731860637,324e236a83b645a5e746604f0d7f9cee
...,...,...,...,...,...
715299,3dbeb7f4350afdacfe4ebbf3d314a6e3,67682a4ad52115434285d25cdad55a91,c489727635800ce9fa0a5d9f10db0c52,8a2fc1a4b86c6b79a6186ee6b54d79e0,af894be6f2cec95b9de30b0a128c83bd
715308,34af8c382bee0cb49c80aa94a3d832d7,8c3fd93c634c05315953692ae207a963,5f8ad3ce1ad2549f241192c6428a9421,02c74a524b6c0883e7200ef4f1ac21ed,da341b2b1cd5b1caf4d604d9766a04d9
715309,b61db567d0f80c97ed0575c60f5de893,beca57431ebd106922926e35511600c2,71976bda1bbdbc1903e18f73e98e7753,35bf614519bd2c2e2e363d3696536c43,5fffdf9f1d562b01a4d4db033e31658e
715318,336ffd7f841c91e7cff653a176a63ae8,f71e4ef635d80a0357f3c4bb1cf4be21,019bf57f7e2694ce94d8c8b15a7883b5,09e9bb67129b50a13f6eb0c352715e15,3888e64ba8a21cbf92199d49d37563d5
